In [64]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math as ma
import sys
np.seterr(divide='ignore', invalid='ignore')
iris = datasets.load_iris()

In [76]:
class bt: #Binary tree class
    def __init__(self, entropy,lvl,split_feature,gain,cls_name): #Constructor
        self.split_feature = split_feature 
        self.entropy = entropy
        self.gain = gain
        self.lvl = lvl
        self.cls_name = cls_name
        self.right = None
        self.left = None
    
def printBinaryTree(root, Node): #Printing the Binary Tree
    if (root == None): # Base case
        return #Return when we reach the root to be None
    print(Node) 
    print("Level :- " ,root.lvl)
    print("Entropy :- " ,root.entropy)
    print("Split_Feature :- ",root.split_feature)
    print("Gain :- ",root.gain)
    print("Class Name :- ",root.cls_name)
    print()
    
    printBinaryTree(root.left, "Left Node") #Left node recurssion call
    printBinaryTree(root.right, "Right Node") #Right node recurssion call

In [66]:
df = pd.DataFrame(iris.data)
bf = pd.DataFrame(iris.target)
df.columns = iris.feature_names
print(iris.target_names)

['setosa' 'versicolor' 'virginica']


In [67]:
def count(y):
    y = np.array(y)
    setosaCount = (y == 0).sum()
    versicolorCount = (y == 1).sum()
    virginicaCount = (y == 2).sum()
    return setosaCount, versicolorCount, virginicaCount

In [68]:
def entropy(y):
    setosa, versicolor, virginica = count(y) #Count of individual flowers
    total = setosa + versicolor + virginica #Total of every flower
    d1 = setosa/total # Accuracy of each flower
    d2 = versicolor/total #Accuracy
    d3 = virginica/total #Accuracy
    
    returnValue = 0 #Initialized the return value
    if(d1 != 0): #Condition to avoid error on log(0)
        returnValue += d1 * ma.log(d1, 2) #Adding the entropy of each result
    if(d2 != 0): #Condition to avoid error on log(0)
        returnValue += d2 * ma.log(d2, 2) #Adding the entropy of each result
    if(d3 != 0): #Condition to avoid error on log(0)
        returnValue += d3 * ma.log(d3, 2) #Adding the entropy of each result
    
    return -returnValue #Totol Entropy Negative sign as per formula to make entropy positive

In [69]:
def split(y1, y2, y):
    p1 = len(y1) / len(y) #Getting accuracy of split 1
    p2 = len(y2) / len(y) #Accuracy of split 2
    
    result = 0 #initialize the result variable with 0 which gives split info
    if(p1 != 0): # Condition to avoid the log(0) error
        result += p1 * ma.log(p1, 2) #Getting split info for each split and adding to var result
    if(p2 != 0): # Condition to avoid the log(0) error
        result += p2 * ma.log(p2, 2) #Getting split info for each split and adding to var result
    return -result #returns the split info

In [70]:
def featureGain(x, y, feature):
    data = np.array(x[feature]) # Creating numpy array of particular features
    tempGain = -1000
    splitValue = 0
    for i in range(1, len(x)): #Iterating through all the feature values
        midPoint = (data[i - 1] + data[i]) / 2 # Finding middle point of all the values
        
        #x_split_1 = data[data > midPoint] # Features greater than mid point
        #x_split_2 = data[data <= midPoint] #Features less than or equal to mid-point
        y_split_1 = y[x[feature] > midPoint] #Results of features greater than midPoint
        y_split_2 = y[x[feature] <= midPoint] # Results of Features less than or equal to mid-point

        finalEntropy = 0
        initialEntropy = entropy(y) #Entropy of root node of each split
        
        entropy_1 = entropy(y_split_1) #Entropy of Split 1
        entropy_2 = entropy(y_split_2) #Entropy of split 2
        
        finalEntropy += entropy_1 * len(y_split_1)/len(y)  
        finalEntropy+= entropy_2 * len(y_split_2)/len(y)  
        infoGain = initialEntropy - finalEntropy #Information Gain
        splitInfo = split(y_split_1, y_split_2, y) #Getting the split Info
        gainRatio = (infoGain / splitInfo) #Calculating Gain Ratio
        
        if(tempGain < gainRatio): #When we find new gain Ratio of new splid greater than previous split's gain ratio
            tempGain = gainRatio #We update previous gain Ratio with better gain Ratio
            splitValue = midPoint #We update the mid or split point with new one which gives better gain ratio
    return tempGain, splitValue #Returning best gain ratio and split Value

In [74]:
def dt(x, y, features, level):
    leftFeatures = features.copy()
    noFeatures = len(features) #Calculate number of Features
    totalData = len(x) 
    setosa, versicolor, virginica = count(y) #Count of Individual flowers
    maxClass = max(setosa, versicolor, virginica) #Class with maximum frequency
    if(setosa == maxClass): #if setosa frequency is highest
        className = "Setosa"
    elif(virginica == maxClass): #if virginica frequency is highest
        className = "Virginica"
    else:
        className = "versicolor" #if versicolor frequency is highest
        
    if(setosa == totalData or versicolor == totalData or virginica == totalData or noFeatures == 0): #Coding of leaf Node
        root = bt(entropy(y), level, "Can't Split Reached Leaf Node", 0, className)
        return root #Return when we reach the leaf node
    else:
        maxGain = -1 #When we do not reach the reaf node, we split so initialse the maximum gain
    
        split = 0 #Initializing the split value
        indexValue = 0
    
        for i in features: #Iterating all the features to get maximum gain
            tempGain, tempSplit = featureGain(x, y, i) #Getting best gain and split value for each feature
            
            if(maxGain < tempGain): #If the initial gain is less than new gain update the initial gain
                maxGain = tempGain #Update the gain
                split = tempSplit #Update the split value
                featureSplit = i #Updating with best split feature
                used = indexValue #Index of the features to be used to split for now
            indexValue += 1 # Index of the feature we are comparing right now
        root = bt(entropy(y), level, featureSplit, maxGain, className) #creating Binsry Tree
        leftFeatures.pop(used) #Deleting the feature once used
    
    #After getting the best split value with maximum gain feature.
    
        x1 = x[x[featureSplit] > split] #When feture greater than the split value
        x2 = x[x[featureSplit] <= split] #Features less than or equal to split value
        y1 = y[x[featureSplit] > split] # Outcomes of features greater than split value
        y2 = y[x[featureSplit] <= split] #Outcomes of features less than or euqal to split value
    
        root.left = dt(x1, y1, leftFeatures, level + 1) #Recurssion call
        root.right = dt(x2, y2, leftFeatures, level + 1) #Recursion Call
        
        return root

In [75]:
 root = dt(df, bf, list(df.columns), 0)

In [73]:
printBinaryTree(root, "Root Node")

Root Node
Level :-  0
Entropy :-  1.584962500721156
Split_Feature :-  petal width (cm)
Gain :-  0.9999999999999999
Class Name :-  Setosa

Left Node
Level :-  1
Entropy :-  1.0
Split_Feature :-  petal length (cm)
Gain :-  0.6621582046482519
Class Name :-  Virginica

Left Node
Level :-  2
Entropy :-  0.4971677614160753
Split_Feature :-  sepal length (cm)
Gain :-  0.05463893158092842
Class Name :-  Virginica

Left Node
Level :-  3
Entropy :-  -0.0
Split_Feature :-  Can't Split Reached Leaf Node
Gain :-  0
Class Name :-  Virginica

Right Node
Level :-  3
Entropy :-  0.5830194167347008
Split_Feature :-  sepal width (cm)
Gain :-  0.0519480903515746
Class Name :-  Virginica

Left Node
Level :-  4
Entropy :-  -0.0
Split_Feature :-  Can't Split Reached Leaf Node
Gain :-  0
Class Name :-  Virginica

Right Node
Level :-  4
Entropy :-  0.6292492238560345
Split_Feature :-  Can't Split Reached Leaf Node
Gain :-  0
Class Name :-  Virginica

Right Node
Level :-  2
Entropy :-  0.1537421803287619
Split_